Schritte:

1. Einlesen der Reviews
2. Preprocessing Schritte durchführen
3.

Installieren der benötigten Bibliotheken.

In [121]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [122]:
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)
df = list(parse("../Rumprobieren/data/data.json.gz"))

In [123]:
df[0]["reviewText"]

"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,"

In [124]:
def replace_contractions(text):
    return contractions.fix(text)
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    #words = lemmatize_verbs(words)
    
    return words


In [125]:
liste = []

for i in range(5):
    var = replace_contractions(df[i]["reviewText"])
    var = nltk.word_tokenize(var)
    var = normalize(var)
    liste.append(var)

print(liste[2])

['primary', 'job', 'device', 'block', 'breath', 'would', 'otherwise', 'produce', 'popping', 'sound', 'allowing', 'voice', 'pass', 'noticeable', 'reduction', 'volume', 'high', 'frequencies', 'double', 'cloth', 'filter', 'blocks', 'pops', 'let', 'us', 'voice', 'coloration', 'metal', 'clamp', 'mount', 'attaches', 'mike', 'stand', 'secure', 'enough', 'keep', 'attached', 'goose', 'neck', 'needs', 'little', 'coaxing', 'stay', 'put']


In [126]:
print(liste[2])

['primary', 'job', 'device', 'block', 'breath', 'would', 'otherwise', 'produce', 'popping', 'sound', 'allowing', 'voice', 'pass', 'noticeable', 'reduction', 'volume', 'high', 'frequencies', 'double', 'cloth', 'filter', 'blocks', 'pops', 'let', 'us', 'voice', 'coloration', 'metal', 'clamp', 'mount', 'attaches', 'mike', 'stand', 'secure', 'enough', 'keep', 'attached', 'goose', 'neck', 'needs', 'little', 'coaxing', 'stay', 'put']
